In [ ]:
import warnings

warnings.filterwarnings('ignore')

from app.controllers.directional_trading.macd_bb_v1 import MACDBBV1ControllerConfig

from core.backtesting.optimizer import BacktestingConfig, BaseStrategyConfigGenerator
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from decimal import Decimal


class MACDBBConfigGenerator(BaseStrategyConfigGenerator):
    """
    Strategy configuration generator for PMM Simple optimization.
    """
    async def generate_config(self, trial) -> BacktestingConfig:
        # Controller configuration
        connector_name = "binance_perpetual"
        trading_pair = "BTC-USDT"
        total_amount_quote = 1000
        take_profit = trial.suggest_float("take_profit", 0.01, 0.03, step=0.01)
        stop_loss = trial.suggest_float("stop_loss", 0.01, 0.05, step=0.01)
        trailing_stop_activation_price = trial.suggest_float("trailing_stop_activation_price", 0.005, 0.015, step=0.01)
        trailing_delta_ratio = trial.suggest_float("trailing_delta_ratio", 0.05, 0.1, step=0.01)
        trailing_stop_trailing_delta = trailing_stop_activation_price * trailing_delta_ratio
        time_limit = trial.suggest_int("time_limit", 60 * 60 * 2, 60 * 60 * 4, step=60 * 60)
        cooldown_time = trial.suggest_int("cooldown_time", 60 * 5, 60 * 10, step=60)
        interval = "3m"
        bb_length = trial.suggest_int("bb_length", 50, 150)
        bb_std = trial.suggest_float("bb_std", 0.1, 2.0)
        fast_ma = trial.suggest_int("fast_ma", 10, 50)
        slow_ma = trial.suggest_int("slow_ma", 51, 150)
        signal_ma = trial.suggest_int("signal_ma", 5, 20)


        # Creating the instance of the configuration and the controller
        config = MACDBBV1ControllerConfig(
            connector_name=connector_name,
            trading_pair=trading_pair,
            total_amount_quote=Decimal(total_amount_quote),
            take_profit=Decimal(take_profit),
            stop_loss=Decimal(stop_loss),
            trailing_stop=TrailingStop(activation_price=Decimal(trailing_stop_activation_price), trailing_delta=Decimal(trailing_stop_trailing_delta)),
            time_limit=time_limit,
            cooldown_time=cooldown_time,
            candles_connector=connector_name,
            candles_trading_pair=trading_pair,
            interval=interval,
            bb_length=bb_length,
            bb_std=bb_std,
            macd_fast=fast_ma,
            macd_slow=slow_ma,
            macd_signal=signal_ma,
        )

        # Return the configuration encapsulated in BacktestingConfig
        return BacktestingConfig(config=config, start=self.start, end=self.end)

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer
import datetime

start_date = datetime.datetime(2025, 8, 1)
end_date = datetime.datetime(2025, 9, 1)
config_generator = MACDBBConfigGenerator(start_date=start_date, end_date=end_date)

optimizer = StrategyOptimizer()
await optimizer.optimize(
    study_name="macd_bb_with_optimizer",
    config_generator=config_generator,
    n_trials=5,
)

In [ ]:
optimizer.launch_optuna_dashboard()

# check the dashboard running on 8080